### 단어를 시퀀스로 처리하기: 시퀀스 모델 방식

#### 첫 번째 예제

**데이터 다운로드**

In [1]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  20.9M      0  0:00:03  0:00:03 --:--:-- 20.9M


**데이터 준비**

In [2]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**정수 시퀀스 데이터셋 준비하기**

In [3]:
# 텍스트 데이터를 벡터화하여 신경망 모델에 입력하기 위한 전처리 과정
from tensorflow.keras import layers

max_length = 600    #시퀀스의 최대 길이
max_tokens = 20000   #한 데이터에서 사용되는 단어들의 최대 개수(많이 쓰이는 거 위주로 2만개만 쓰겠다)

text_vectorization = layers.TextVectorization(   #텍스트 벡터화 레이어
    max_tokens=max_tokens,
    output_mode="int",   #텍스트 데이터를 정수로 변환
    output_sequence_length=max_length,)
text_vectorization.adapt(text_only_train_ds)   #벡터화 레이어를 데이터셋에 적용

#벡터화된 정수 시퀀스를 데이터셋에 매핑
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델 만들기**

In [2]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64")   #텍스트 시퀀스를 입력으로 받음( 각 단어를 정수로 표현)
embedded = tf.one_hot(inputs, depth=max_tokens)   #원핫 인코딩
x = layers.Bidirectional(layers.LSTM(32))(embedded)   #양방향 LSTM 추가(입력 시퀀스를 앞뒤로 모두 고려)
x = layers.Dropout(0.5)(x)   #드롭아웃 레이어
outputs = layers.Dense(1, activation="sigmoid")(x)   #출력 레이어

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

NameError: ignored

**첫 번째 시퀀스 모델 훈련하기**

In [5]:
callbacks = [ keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")

print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 160s 242ms/step - loss: 0.5645 - accuracy: 0.7035 - val_loss: 0.3933 - val_accuracy: 0.8464
Epoch 2/10
625/625 [==============================] - 154s 246ms/step - loss: 0.3659 - accuracy: 0.8630 - val_loss: 0.3681 - val_accuracy: 0.8488
Epoch 3/10
625/625 [==============================] - 153s 246ms/step - loss: 0.2987 - accuracy: 0.8946 - val_loss: 0.3377 - val_accuracy: 0.8634
Epoch 4/10
625/625 [==============================] - 154s 246ms/step - loss: 0.2504 - accuracy: 0.9121 - val_loss: 0.3014 - val_accuracy: 0.8790
Epoch 5/10
625/625 [==============================] - 153s 245ms/step - loss: 0.2178 - accuracy: 0.9248 - val_loss: 0.3090 - val_accuracy: 0.8834
Epoch 6/10
625/625 [==============================] - 153s 245ms/step - loss: 0.1949 - accuracy: 0.9338 - val_loss: 0.3360 - val_accuracy: 0.8844
Epoch 7/10
625/625 [==============================] - 153s 245ms/step - loss: 0.1739 - accuracy: 0.9434 - val_loss: 0.3296 -

#### 단어 임베딩 이해하기

#### 임베딩 층으로 단어 임베딩 학습하기

**`Embedding` 층 만들기**

In [ ]:
# 임베딩 레이어: 입력 정수 시퀀스 최대값, 임베딩 벡터 크기(차원)을 매개변수로 받음
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

# 텍스트 데이터(고차원 희소벡터)의 각 단어를 고정된 크기의 실수 벡터(저차원 밀집벡터)로 변환
# 단어표현 학습/ 차원축소/ 단어간 유사성(거리, 유사도) 비교/ 전이학습(학습된 모델 사용해 초기 가중치 설정)

**밑바닥부터 훈련하는 `Embedding` 층을 사용한 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)   #임베딩 레이어
x = layers.Bidirectional(layers.LSTM(32))(embedded)   #양방향 LSTM
x = layers.Dropout(0.5)(x)   #드롭아웃 레이어
outputs = layers.Dense(1, activation="sigmoid")(x)   #출력층

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [ keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras",
                                    save_best_only=True)]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.keras")

print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

#### 패딩과 마스킹 이해하기

**마스킹을 활성화한 `Embedding` 층 사용하기**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256,
    mask_zero=True)(inputs)   #입력 시퀀스에서 값이 0인 토큰을 마스킹(모델이 해당 토큰을 무시하도록)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [ keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.keras",
                                    save_best_only=True) ]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_with_masking.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

#### 사전 훈련된 단어 임베딩 사용하기

In [9]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-07-14 02:51:29--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-14 02:51:30--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-14 02:51:30--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**GloVe 단어 임베딩 파일 파싱하기**

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"단어 벡터 개수: {len(embeddings_index)}")

**GloVe 단어 임베딩 행렬 준비하기**

In [ ]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**사전 훈련된 임베딩을 사용하는 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________